In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
from peft import AutoPeftModelForCausalLM


fintune_path = "output_qwen/checkpoint-1000"

model = AutoPeftModelForCausalLM.from_pretrained(
    fintune_path, # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

tokenizer = AutoTokenizer.from_pretrained(
    fintune_path, # path to the output directory
    trust_remote_code=True
)

DEFAULT_SYSTEM_PROMPT = """
你作为海报设计人员，请根据用户输入执行以下任务来理解设计要素：
1. 判断输入是否是一个“有效”的平面设计的请求。
2. 信息匹配：匹配“场景”字段，要求只能且必须在[PC端banner,  手机淘宝banner, 移动端banner, 电商横版海报, 电商全屏海报, 电商竖版海报, 商品主图, 众号封面首图, 公众号封面小图, 横版海报, 每日一签, 竖版海报, 手机海报, 邀请函]中选择一个最匹配的；如果没有最匹配的选项返回None。
3. 信息提取：提取“设计用途”，“主标题”，“关键词”，“颜色”，“人物”，“时间”，“地点”；如果没有匹配的选项，则返回None;
“设计用途”为海报设计的使用场景、期望得到的效果。比如营销口红产品、庆祝圣诞节等。
返回结果必须是字典格式。

例子1：用户输入："生成一个婚礼贺请柬，12月25日举办，李颖和刘涛的婚礼，红色"。回答：{"有效":True, "场景": "邀请函","颜色": "红色", "设计用途":"", "人物":"李颖和刘涛", "时间":"12月25日", 关键词:"婚礼贺请柬"}
例子2：用户输入："帮我生成一个日签，粉红色主题"。 回答：{"有效":True, "场景": "每日一签", "颜色": "粉红", "关键词":"日签 粉红色","设计用途":""}
例子3：用户输入："日签图片，以绿色为主色调，加入春花、蝴蝶等元素，传达清新、明媚的气息，文字积极向上，鼓舞人心，标题：元旦你好"。回答：{"有效":True, "场景": "每日一签", "颜色": "绿色", "设计用途":"传达清新、明媚的气息，文字积极向上，鼓舞人心的设计", "关键词":"日签 春花 蝴蝶", "主标题":"元旦你好"}
例子4：用户输入："招聘海报，吸引人才，展示公司优势，突出职位特点，简洁明了，标题是小可爱英语班"。回答：{"有效":True, "场景": "竖版海报", "颜色": None, "设计用途":"吸引人才，展示公司优势，突出职位特点", "关键词":"公司 人才", "主标题":"小可爱英语班"}
例子5：用户输入："你好" 回答：{"有效":False}

"""

# response, history = model.chat(tokenizer, "用户输入：主图", history=None, system=DEFAULT_SYSTEM_PROMPT)
# print(response)

/bigdisk/lax/weshiz/gitlab_workspace/Qwen/qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model is automatically converting to fp16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.19it/s]


In [5]:
response, history = model.chat(tokenizer, "用户输入：小明设计的一个Apple年会的宣传彩页", history=None, system=DEFAULT_SYSTEM_PROMPT)
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (562 > 512). Running this sequence through the model will result in indexing errors


{'有效': False, '场景': None, '颜色': None, '设计用途': None, '人物': '小明', '时间': None, '地点': None, '关键词': 'Apple年会 宣传彩页', '主标题': None}


In [6]:
response, history = model.chat(tokenizer, "用户输入：小明设计的一个Apple年会的宣传彩页", history=None, system="")
print(response)

{'有效': True, '场景': 'PC端banner', '颜色': None, '设计用途': 'Apple年会的宣传彩页', '人物': '小明', '时间': None, '地点': None, '关键词': 'Apple 年会 宣传彩页', '主标题': None}


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
from peft import PeftMixedModel


# Model names: "Qwen/Qwen-7B-Chat", "Qwen/Qwen-14B-Chat"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B-Chat", trust_remote_code=True, fp16=True)

base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-7B-Chat",
    device_map="auto",
    trust_remote_code=True
).eval()

fintune_path = "output_qwen/checkpoint-1000"
peft_model = PeftMixedModel.from_pretrained(base_model, fintune_path)
# peft_model.load_adapter(<path_to_adapter2>, adapter_name="other")
# peft_model.set_adapter(["default", "other"])


/bigdisk/lax/weshiz/gitlab_workspace/Qwen/qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model is automatically converting to fp16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.06it/s]


RuntimeError: Error(s) in loading state_dict for PeftMixedModel:
	size mismatch for base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight: copying a param with shape torch.Size([8, 2048]) from checkpoint, the shape in current model is torch.Size([8, 4096]).
	size mismatch for base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight: copying a param with shape torch.Size([6144, 8]) from checkpoint, the shape in current model is torch.Size([12288, 8]).
	size mismatch for base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight: copying a param with shape torch.Size([8, 2048]) from checkpoint, the shape in current model is torch.Size([8, 4096]).
	size mismatch for base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight: copying a param with shape torch.Size([2048, 8]) from checkpoint, the shape in current model is torch.Size([4096, 8]).
	size mismatch for base_model.model.transformer.h.0.mlp.w1.lora_A.default.weight: copying a param with shape t